#@title 🔑 Clusterizza e Misura (speciale AST 2025)

[![Licenza MIT](https://img.shields.io/badge/License-MIT-green.svg)](https://opensource.org/licenses/MIT)

In [1]:
# FONDAMENTALE!
# Installiamo in memoria le librerie necessarie
# NB: nelle macchine virtuali di colab è necessario installare ogni volta le librerie se la macchina è stata riavviata o si è auto-disattivata
!pip install requests pandas -q
print("Dipendenze installate.")


Dipendenze installate.


In [2]:
#@title 🔑 Inserisci la chiave API di OpenRouter (o usa i Segreti di Colab)
#@markdown ---
#@markdown **Questa cella ti permette di fornire la tua chiave API di OpenRouter.**
#@markdown
#@markdown **Opzione 1 (Consigliata & Sicura): Usa i Segreti di Colab**
#@markdown 1. Clicca sull'icona '🔑' (Chiave) nella barra laterale sinistra.
#@markdown 2. Aggiungi un nuovo segreto chiamato `OPENROUTER_API_KEY`.
#@markdown 3. Incolla la tua chiave API come valore e abilita "Accesso al notebook".
#@markdown 4. **Lascia il campo di input qui sotto VUOTO.** Lo script userà automaticamente il segreto.
#@markdown ---
#@markdown **Opzione 2 (Meno Sicura - Incolla Direttamente):**
#@markdown Se preferisci non usare i Segreti di Colab, incolla direttamente la tua chiave API nel campo qui sotto.
#@markdown **Nota:** Chiunque condivida questo notebook con te e esegua questa cella potrebbe vedere la chiave se ispeziona l'output salvato.
api_key_from_form = "la_tua_chiave_api" #@param {type:"string"}
#@markdown ---
print("Campo di inserimento della chiave API elaborato. L'assegnazione della chiave avviene nella cella successiva.")


Campo di inserimento della chiave API elaborato. L'assegnazione della chiave avviene nella cella successiva.


In [3]:
#@title Prompts

# il classification prompt è quello che serve per classificare le query in base alla categoria e alla sotto categoria
# il similarity prompt è quello che serve per misurare la somiglianza semantica tra il risultato dell'LLM e la ground truth letta da file

# se dovete cambiare i prompt, potete farlo qui sotto, per ottenere un risultato soddisfacente dovete definire categorie e sotto categorie che siano appropriate per il vostro dataset

CLASSIFICATION_PROMPT_TEMPLATE = """Classify a series of queries related to tourism in Italy for the year 2025.
Assign each query only one of the most appropriate category from the list provided, then assign Only one of the most appropriate subcategory from the list.
If the term is not Italian, interpret the meaning and choose the closest category and subcategory listed.
Categories
	-	Cantine
	-	Degustazione formaggi
	-	Degustazione grappe
	-	Degustazione miele
	-	Degustazione vini
	-	Frantoi
	-	Servizi
	-	Turismo enogastronomico
Subcategories
	-	Franciacorta
	-	Lago di Garda
	-	Trentino Alto Adige
	-	Generico
	-	Online
Output format
category,subcategory
(do not add additional context!)


Examples, (ground truth)
	- cantina franciacorta visita > Cantine,Franciacorta
	- degustazione vini in bicicletta > Degustazione vini,Generico
	- degustazione vini sirmione > Degustazione vini,Lago di Garda
	- weekend enogastronomici > Turismo enogastronomico,Generico
query: {input_text}
Answer:
"""

SIMILARITY_PROMPT_TEMPLATE = """
Abbiamo fatto la classificazione di una query.
Ti fornisco il risultato ottenuto dall'LLM e la classificazione corretta (ground truth).
Misura la somiglianza semantica tra questi due testi.
Voglio un punteggio per capire quanto il risultato dell'LLM corrisponde alla ground truth.
Esprimi **solo** un punteggio numerico compreso tra 0 (completamente dissimile) e 100 (semanticamente identico), senza aggiungere alcuna spiegazione o testo aggiuntivo.

Ground Truth: "{ground_truth_text}"
Risultato LLM: "{llm_result_text}"

Punteggio (0-100):
"""


In [4]:
#@title 3.Importiamo le chiavi API e i settaggi

# questo passaggio è per rendere il sistema più robusto e sicuro, ma per dei testi privati è sufficiente inserire la chiave API direttamente nella cella 2

import os
import sys
import requests
import pandas as pd
import time
import re
from google.colab import userdata # For Colab secrets
from google.colab import files    # For file uploads/downloads

# --- Determine the Final API Key ---
FINAL_API_KEY = None
api_key_source = "None"

# 1. Try Colab Secrets first
try:
    key_from_secrets = userdata.get('OPENROUTER_API_KEY')
    if key_from_secrets:
        FINAL_API_KEY = key_from_secrets
        api_key_source = "Colab Secrets"
        print("✅ API Key loaded successfully from Colab Secrets.")
    else:
         print("ⓘ API Key Secret 'OPENROUTER_API_KEY' found but is empty. Checking form input...")
except userdata.SecretNotFoundError:
    print("ⓘ API Key Secret 'OPENROUTER_API_KEY' not found. Checking form input...")
except Exception as e:
    print(f"⚠️ Error accessing Colab Secrets: {e}. Checking form input...")

# 2. Fallback to form input if secrets didn't yield a key
if FINAL_API_KEY is None:
    if 'api_key_from_form' in locals():
        key_input_stripped = api_key_from_form.strip()
        if key_input_stripped:
            FINAL_API_KEY = key_input_stripped
            api_key_source = "Direct Input Form"
            print("✅ API Key loaded from the direct input form field.")
            print("   ⚠️ Warning: Using the form field is less secure than Colab Secrets.")
        else:
             print("ⓘ Direct input form field is also empty.")
    else:
         print("Error: api_key_from_form variable not found. Ensure Cell 2 ran.")

# 3. Final Check
if FINAL_API_KEY:
    print(f"🔑 Using API Key obtained from: {api_key_source}")
else:
    print("❌ ERROR: OpenRouter API Key is MISSING.")
    print("   Please provide it via Colab Secrets or the form in Cell 2.")

# --- Istruzioni per il caricamento del file ---
print("\n📄 Carica il tuo file CSV di input (ad esempio, 'input.csv').")
print("   Deve contenere obbligatoriamente le colonne chiamate 'input' e 'ground_truth'.")
print("   La colonna 'input' deve contenere le query da classificare.")
print("   La colonna 'ground_truth' deve contenere le categorie e sotto categorie identificate dal professionista (vedi prompt).")
print("   ✅ Usa la scheda 'File' (icona a forma di cartella) nel pannello a sinistra per caricare il file.")


ⓘ API Key Secret 'OPENROUTER_API_KEY' not found. Checking form input...
✅ API Key loaded from the direct input form field.
   ⚠️ Warning: Using the form field is less secure than Colab Secrets.
🔑 Using API Key obtained from: Direct Input Form

📄 Carica il tuo file CSV di input (ad esempio, 'input.csv').
   Deve contenere obbligatoriamente le colonne chiamate 'input' e 'ground_truth'.
   La colonna 'input' deve contenere le query da classificare.
   La colonna 'ground_truth' deve contenere le categorie e sotto categorie identificate dal professionista (vedi prompt).
   ✅ Usa la scheda 'File' (icona a forma di cartella) nel pannello a sinistra per caricare il file.


In [5]:
#@title 4. Configuriamo i modelli, i prompt, i file e la pausa in secondi tra una query e l'altra
# inserite il modello classificatore e quello per la misura della somiglianza semantica
# inserite il percorso del file csv di input e di output
# inserite il tempo di pausa tra una query e l'altra


# --- LLM Models ---
CLASSIFICATION_MODEL = "openrouter/horizon-beta" #@param {type:"string"}
SIMILARITY_MODEL = "openai/gpt-4o-mini" #@param {type:"string"}


# --- File Paths ---
# Assumes files are uploaded to the root Colab directory '/content/'
INPUT_CSV_PATH = "/content/test-300-martino.csv" #@param {type:"string"}
OUTPUT_CSV_PATH = "/content/evaluation_results_horizonbeta.csv" #@param {type:"string"}

# --- Processing Delay ---
DELAY_SECONDS = 0.3 #@param {type:"number"}

# --- API Endpoint (Usually fixed) ---
OPENROUTER_API_ENDPOINT = "https://openrouter.ai/api/v1/chat/completions"

# --- Validation & Summary ---
errors = []
if not CLASSIFICATION_MODEL: errors.append("CLASSIFICATION_MODEL non può essere vuoto.")
if not SIMILARITY_MODEL: errors.append("SIMILARITY_MODEL non può essere vuoto.")
if "{input_text}" not in CLASSIFICATION_PROMPT_TEMPLATE: errors.append("CLASSIFICATION_PROMPT_TEMPLATE manca di '{input_text}'.")
if "{ground_truth_text}" not in SIMILARITY_PROMPT_TEMPLATE: errors.append("SIMILARITY_PROMPT_TEMPLATE manca di '{ground_truth_text}'.")
if "{llm_result_text}" not in SIMILARITY_PROMPT_TEMPLATE: errors.append("SIMILARITY_PROMPT_TEMPLATE manca di '{llm_result_text}'.")
if not INPUT_CSV_PATH: errors.append("INPUT_CSV_PATH non può essere vuoto.")
if not OUTPUT_CSV_PATH: errors.append("OUTPUT_CSV_PATH non può essere vuoto.")
if DELAY_SECONDS < 0: errors.append("DELAY_SECONDS non può essere negativo.")

if errors:
    print("❌ Errori nella configurazione:")
    for err in errors:
        print(f"   - {err}")
else:
    print("✅ La configurazione sembra corretta.")
    print("\n--- Impostazioni attuali ---")
    print(f"Modello di classificazione: {CLASSIFICATION_MODEL}")
    print(f"Modello di similarità: {SIMILARITY_MODEL}")
    print(f"CSV di input: {INPUT_CSV_PATH}")
    print(f"CSV di output: {OUTPUT_CSV_PATH}")
    print(f"Attesa per riga: {DELAY_SECONDS}s")
    # Visualizzare i prompt potrebbe essere lungo, valuta se commentare questa parte se necessario
    # print("Prompt di classificazione:\n```\n" + CLASSIFICATION_PROMPT_TEMPLATE + "\n```")
    # print("Prompt di similarità:\n```\n" + SIMILARITY_PROMPT_TEMPLATE + "\n```")
    print("Prompt impostati (visualizzabili nel form sopra).")


✅ La configurazione sembra corretta.

--- Impostazioni attuali ---
Modello di classificazione: openrouter/horizon-beta
Modello di similarità: openai/gpt-4o-mini
CSV di input: /content/test-300-martino.csv
CSV di output: /content/evaluation_results_horizonbeta.csv
Attesa per riga: 0.3s
Prompt impostati (visualizzabili nel form sopra).


In [6]:
#@title 5. Prepariamo le funzioni per chiamare l'API di OpenRouter.
## attenzione: alcuni valori sono definiti in questa cella, ma possono essere modificati manualmente.

# lunghezza massima del testo di output per il modello di classificazione
MAX_TOKENS_CLASSIFICATION = 500
# lunghezza massima del testo di output per il modello di similarità semantica
MAX_TOKENS_SIMILARITY = 550
# temperatura per il modello di classificazione
TEMPERATURE_CLASSIFICATION = 0.7
# temperatura per il modello di similarità semantica
TEMPERATURE_SIMILARITY = 0.1

def call_openrouter_llm(messages, model_name, api_key, purpose="task", timeout=60):
    """Funzione generica per chiamare l'API di OpenRouter Chat Completions."""
    if not api_key: # Check moved inside
        print(f"\n❌ Error ({purpose}): la chiave API di OpenRouter è mancante.")
        return None

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        # Add Referer/Title if needed for OpenRouter tracking
        "HTTP-Referer": "https://colab.research.google.com/",
        "X-Title": "Colab LLM Eval", # questo è per tracciare l'utilizzo dell'app di colab dentro openrouter.com
    }
    max_tokens = MAX_TOKENS_CLASSIFICATION if purpose == "classification" else MAX_TOKENS_SIMILARITY
    temperature = TEMPERATURE_CLASSIFICATION if purpose == "classification" else TEMPERATURE_SIMILARITY

    data = {"model": model_name, "messages": messages, "max_tokens": max_tokens, "temperature": temperature}

    try:
        response = requests.post(OPENROUTER_API_ENDPOINT, headers=headers, json=data, timeout=timeout)
        response.raise_for_status()
        result = response.json()
        choices = result.get("choices")
        if choices and len(choices) > 0 and choices[0].get("message") and choices[0]["message"].get("content"):
            return choices[0]["message"]["content"].strip()
        else:
            print(f"\n⚠️ Warning ({purpose}): non è stato possibile estrarre il contenuto. Response: {result}")
            return None
    except requests.exceptions.RequestException as e:
        error_msg = f"\n❌ Error ({purpose}) API call to {model_name}: {e}"
        if hasattr(e, 'response') and e.response is not None:
            error_msg += f". Status: {e.response.status_code}, Body: {e.response.text}"
        print(error_msg)
        return None
    except Exception as e:
        print(f"\n❌ Errore imprevisto durante la chiamata API {purpose} ({model_name}): {e}")
        return None

def get_llm_classification(query_text, model_name, api_key):
    """Ottiene la classificazione usando il template predefinito."""
    try:
        # Access global template defined in Cell 4
        prompt = CLASSIFICATION_PROMPT_TEMPLATE.format(input_text=query_text)
    except KeyError:
        print(f"\n❌ Error: CLASSIFICATION_PROMPT_TEMPLATE manca di '{{input_text}}'. Controlla la cella 4.")
        return None
    except Exception as e:
        print(f"\n❌ Errore nel formattare il prompt di classificazione: {e}")
        return None
    messages = [{"role": "user", "content": prompt}]
    return call_openrouter_llm(messages, model_name, api_key, purpose="classification")

def get_semantic_similarity_score(llm_output, ground_truth, model_name, api_key):
    """calcola la similarità semantica tra il risultato dell'LLM e la ground truth"""
    if not isinstance(llm_output, str) or not isinstance(ground_truth, str) or not llm_output or not ground_truth:
        return 0.0
    try:
        # Access global template defined in Cell 4
        prompt = SIMILARITY_PROMPT_TEMPLATE.format(ground_truth_text=ground_truth, llm_result_text=llm_output)
    except KeyError:
        print(f"\n❌ Errore: SIMILARITY_PROMPT_TEMPLATE manca di placeholder(s). Controlla la cella 4.")
        return 0.0
    except Exception as e:
        print(f"\n❌ Errore nel formattare il prompt di similarità: {e}")
        return 0.0
    messages = [{"role": "user", "content": prompt}]
    raw_score_text = call_openrouter_llm(messages, model_name, api_key, purpose="similarity")
    if raw_score_text is None:
        print("   La misurazione della similarità semantica è fallita (errore API).")
        return 0.0
    # Robust Score Parsing
    try:
        score = float(raw_score_text.strip())
        return max(0.0, min(100.0, score))
    except ValueError:
        match = re.search(r'\b(\d{1,3}(?:\.\d+)?)\b', raw_score_text)
        if match:
            try:
                score = float(match.group(1))
                print(f"   [Sim Score Parsed: {score} from '{raw_score_text}']", end='')
                return max(0.0, min(100.0, score))
            except ValueError: pass
        print(f"   La misurazione della similarità semantica è fallita: non è stato possibile estrarre il numero da '{raw_score_text}'.")
        return 0.0

print("✅ Helper functions defined.")


✅ Helper functions defined.


In [7]:
#@title 6. Define Main Evaluation Logic

def run_evaluation(input_csv_path, output_csv_path, class_model, sim_model, api_key, delay):
    """
    Esegue il ciclo completo di valutazione utilizzando la configurazione definita nelle celle precedenti.
    """
    print(f"\n🚀 Inizio il processo di valutazione...")
    print("---")

    # --- Validate API Key ---
    if not api_key:
        print("❌ CRITICAL ERROR: la chiave API è mancante. Non posso procedere.")
        return None # Stop execution

    # --- Load Input Data ---
    try:
        input_df = pd.read_csv(input_csv_path)
        required_cols = ['input', 'ground_truth']
        input_df.columns = [col.lower().strip() for col in input_df.columns] # Normalize column names
        missing_cols = [col for col in required_cols if col not in input_df.columns]
        if missing_cols:
            raise ValueError(f"Il file CSV '{input_csv_path}' mancano delle colonne richieste: {', '.join(missing_cols)}")
        print(f"👍 Caricato {len(input_df)} righe da '{input_csv_path}'. Colonne: {list(input_df.columns)}")
    except FileNotFoundError:
        print(f"❌ Error: il file non è stato trovato in '{input_csv_path}'. Hai caricato il file?")
        return None
    except ValueError as ve:
        print(f"❌ Error: {ve}")
        return None
    except Exception as e:
        print(f"❌ Errore nel caricamento del file CSV '{input_csv_path}': {e}")
        return None

    # --- Run Evaluations ---
    results_list = []
    total_items = len(input_df)

    for index, row in input_df.iterrows():
        item_num = index + 1
        print(f"🔄 Processo query {item_num}/{total_items}: ", end='')

        original_query = str(row.get('input', '')).strip()
        ground_truth = str(row.get('ground_truth', '')).strip()

        if not original_query:
            print(" Saltato - query vuota.")
            llm_classification = "INPUT_ERROR"
            exact_match = False
            similarity_score = 0.0
        else:
            # 1. Classification
            print(" Classificazione...", end='')
            llm_classification = get_llm_classification(original_query, class_model, api_key)
            if llm_classification is None:
                print(" Classificazione fallita.", end='')
                llm_classification = "CLASSIFICATION_API_ERROR"
                exact_match = False
                similarity_score = 0.0
            else:
                llm_classification = str(llm_classification)
                print(f" Got '{llm_classification}'.", end='')
                # 2. Exact Match
                exact_match = (llm_classification == ground_truth)
                print(f" Match: {exact_match}.", end='')
                # 3. Similarity Score
                print(" Scoring Similarity...", end='')
                similarity_score = get_semantic_similarity_score(llm_classification, ground_truth, sim_model, api_key)
                print(f" Score: {similarity_score:.2f}.", end='')

        # Store Result
        results_list.append({
            'original_query': original_query,
            'ground_truth': ground_truth,
            'classification': llm_classification,
            'match': exact_match,
            'semantic_similarity': round(similarity_score, 2)
        })
        print("") # Newline after processing each item
        time.sleep(delay) # Apply delay

    # --- Save Results ---
    if results_list:
        results_df = pd.DataFrame(results_list)
        try:
            results_df.to_csv(output_csv_path, index=False, encoding='utf-8')
            print(f"\n--- ✅ Valutazione Completata ---")
            print(f"Risultati salvati in '{output_csv_path}'. Puoi scaricarli dalla scheda Files o il browser dovrebbe proporre di scaricare il file.")
            # --- Calculate and Print Statistics ---
            total_queries = len(results_df)
            exact_matches = results_df['match'].sum()
            partial_matches = total_queries - exact_matches
            avg_similarity = results_df['semantic_similarity'].mean()

            print("\n--- Statistiche di valutazione ---")
            print(f"Totale query processate: {total_queries}")
            print(f"Query con corrispondenza al 100%: {exact_matches}")
            print(f"Query con corrispondenza <100%: {partial_matches}")
            print(f"Stima Similarità semantica media: {avg_similarity:.2f}")
            # restituisce l'output
            return output_csv_path # Return filename on success

        except Exception as e:
            print(f"\n❌ Errore nel salvataggio dei risultati in '{output_csv_path}': {e}")
            return None
    else:
        print("\n--- ⚠️ Valutazione Completata ---")
        print("Attenzione: nessun risultato generato. Controlla il file di input e i log.")
        return None

print("✅ Funzione principale di valutazione 'run_evaluation' pronta ad essere chiamata.")

✅ Funzione principale di valutazione 'run_evaluation' pronta ad essere chiamata.


In [8]:
#@title 7. ▶️ Eseguiamo la valutazione

# importiamo il time e facciamo partire il cronomentro
import time # Import the time module
start_time = time.time() # Use time.time() to get the timestamp

# controlliamo che tutto sia pronto per l'esecuzione
# --- Final Check Before Running ---
if 'FINAL_API_KEY' in locals() and FINAL_API_KEY and \
   'CLASSIFICATION_MODEL' in locals() and CLASSIFICATION_MODEL and \
   'SIMILARITY_MODEL' in locals() and SIMILARITY_MODEL and \
   'INPUT_CSV_PATH' in locals() and INPUT_CSV_PATH and \
   'OUTPUT_CSV_PATH' in locals() and OUTPUT_CSV_PATH and \
   'DELAY_SECONDS' in locals() and DELAY_SECONDS >= 0:

    print("Controlli pre-esecuzione passati. Inizio valutazione...")
    #print(CLASSIFICATION_PROMPT_TEMPLATE)
    #print(SIMILARITY_PROMPT_TEMPLATE)

    # --- Call the main function ---
    output_file = run_evaluation(
        input_csv_path=INPUT_CSV_PATH,
        output_csv_path=OUTPUT_CSV_PATH,
        class_model=CLASSIFICATION_MODEL,
        sim_model=SIMILARITY_MODEL,
        api_key=FINAL_API_KEY,
        delay=DELAY_SECONDS
    )

    # --- Optional: Offer download link ---
    if output_file and os.path.exists(output_file):
        try:
            print(f"\n⬇️ Tentativo di attivare il download per '{os.path.basename(output_file)}'...")
            files.download(output_file)
        except Exception as e:
            print(f"   Non è stato possibile attivare il download automatico: {e}")
            print(f"   Scarica '{output_file}' manualmente dalla scheda Files.")
    elif output_file:
         print(f"\nNota: il file di output '{output_file}' era previsto ma non trovato per il download.")

else:
    print("\n❌ Esecuzione annullata: una o più configurazioni critiche mancano.")
    print("   Assicurati di avere:")
    print("   1. Inserito la chiave API (cella 2 & 3).")
    print("   2. Configurato i modelli, i percorsi dei file e i prompt (cella 4).")
    print("   3. Caricato il file CSV di input, attenzione ai nomi delle colonne!")
    print("   Riavvia le celle sopra se necessario.")

#@title 🔔 Final Notification Sound

import requests
from IPython.display import Audio, display

try:
    # Using a more common format like MP3 might be slightly more compatible, but OGG usually works.
    url = "https://upload.wikimedia.org/wikipedia/commons/0/0c/Meow_domestic_cat.ogg"
    #url = "https://interactive-examples.mdn.mozilla.net/media/cc0-audio/t-rex-roar.mp3" # Example MP3

    # Define a custom User-Agent header
    headers = {
        "User-Agent": "MyColabScript/1.0 (https://colab.research.google.com/; MyEmail@example.com)"
    }

    audio_response = requests.get(url, headers=headers, timeout=10) # Pass headers to the request
    audio_response.raise_for_status() # Check for HTTP errors
    audio_content = audio_response.content

    print("segnale audio...")
    # Explicitly display the Audio object
    display(Audio(audio_content, autoplay=True)) # Added autoplay=True

except requests.exceptions.RequestException as e:
    print(f"❌ Errore nel recupero del segnale audio: {e}")
except Exception as e:
    print(f"❌ Errore nel riprodurre il segnale audio: {e}")

# fermiamo il timer e stampiamo i secondi
end_time = time.time() # Use time.time() to get the timestamp
elapsed_time = end_time - start_time
print(f"Tempo di esecuzione: {elapsed_time:.2f} secondi")
# e lo stampiamo anche in minuti
print(f"Tempo di esecuzione: {elapsed_time/60:.2f} minuti")

Controlli pre-esecuzione passati. Inizio valutazione...

🚀 Inizio il processo di valutazione...
---
👍 Caricato 325 righe da '/content/test-300-martino.csv'. Colonne: ['input', 'ground_truth']
🔄 Processo query 1/325:  Classificazione... Got 'Cantine,Franciacorta'. Match: True. Scoring Similarity... Score: 100.00.
🔄 Processo query 2/325:  Classificazione... Got 'Cantine,Franciacorta'. Match: True. Scoring Similarity... Score: 100.00.
🔄 Processo query 3/325:  Classificazione... Got 'Cantine,Franciacorta'. Match: True. Scoring Similarity... Score: 100.00.
🔄 Processo query 4/325:  Classificazione... Got 'Cantine,Franciacorta'. Match: True. Scoring Similarity... Score: 100.00.
🔄 Processo query 5/325:  Classificazione...

KeyboardInterrupt: 